In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import SGD, Adadelta
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import Callback
from tensorflow.keras import backend as K
from sklearn.decomposition import PCA
from tensorflow.keras.models import Model,load_model
import sys
import os

from tensorflow.python.ops import control_flow_ops
from tensorflow.python.ops import math_ops
from tensorflow.python.ops import state_ops
from tensorflow.python.framework import ops
from tensorflow.python.training import optimizer
import tensorflow as tf


import numpy as np
import random
import math
import matplotlib.pyplot as plt


In [2]:
###comment this code, if you want to use your default gpu. I change to CPU becuase
###I get Resource exhausted on my GPU = =.
os.environ['CUDA_VISIBLE_DEVICES'] = ''
###
if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

####

No GPU found


In [3]:
#Config
epochs = 250
batch = 128
data_size = 10000


In [4]:
def trainingData():    
    def target(x):
    	assert( x >= 0 )
    	y = x * math.sin(5*math.pi *x)
    	return y
    
    x = []
    y = []
    for i in range(data_size):
        x.append(random.random())        
    y = [target(i) for i in x]
    
    return x,y

In [ ]:
class NewtonOptimizer(optimizer.Optimizer, loss):
    """Implementation of AddSign.
    See [Bello et. al., 2017](https://arxiv.org/abs/1709.07417)
    @@__init__
    """

    def __init__(self, loss, use_locking=False, name="AddSign"):
        super(NewtonOptimizer, self).__init__(use_locking, name)
        hess = tf.hessians(loss, wrt_variables)
        inv_hess = [tf.matrix_inverse(h) for h in hess]
        self._lr = loss

        
        # Tensor versions of the constructor arguments, created in _prepare().
        self._lr_t = None

      
    def _prepare(self):
        self._lr_t = ops.convert_to_tensor(self._loss, name="learning_rate")


    def _create_slots(self, var_list):
        # Create slots for the first and second moments.
        for v in var_list:
            self._zeros_slot(v, "m", self._name)

    def _apply_dense(self, grad, var):
        self._lr
        self._lr_t = ops.convert_to_tensor(self._lr, name="learning_rate")
        lr_t = math_ops.cast(self._lr_t, var.dtype.base_dtype)
        
        hess = tf.hessians(loss, wrt_variables)
        inv_hess = [tf.matrix_inverse(h) for h in hess]
    
        eps = 1e-7 #cap for moving average
        
        var_update = state_ops.assign_sub(var, lr_t*grad*(1.0+alpha_t*tf.sign(grad)*tf.sign(m_t) ) )
        #Create an op that groups multiple operations
        #When this op finishes, all ops in input have finished
        return control_flow_ops.group(*[var_update, m_t])

    def _apply_sparse(self, grad, var):
        raise NotImplementedError("Sparse gradient updates are not supported.")

In [ ]:
class MY_FUNCTION(object):
    def __init__(self, width = 32, height = 32, channels = 3):
        self.width = width
        self.height = height
        self.channels = channels
        
        self.shape = (self.width, self.height, self.channels)
        self.optimizer = NewtonOptimizer()
        
        self.model = self.__model()
        self.model.compile(loss='categorical_crossentropy', optimizer=self.optimizer)
        
    def __model(self):
        model = Sequential()
        model.add(Dense(10, input_shape=(1,), activation='relu'))
        model.add(Dense(18,  activation='relu'))
        model.add(Dense(15,  activation='relu'))
        model.add(Dense(4,  activation='relu'))
        model.add(Dense(1,  activation='linear')) 
        model.summary()
        
        return model
    
    
    def train(self, x_train, y_train, epochs = 500, batch = 256, collect_interval = 3):
        loss = []
        accuracy = []
        weights = []
        grads = []
        
        for cnt in range(epochs):
            history = self.model.fit(x_train, y_train, batch_size=batch, verbose = 0)
            print("epoch:%d ,loss:%s "%(cnt, history.history['loss']))
            loss.append(history.history['loss'])
            accuracy.append(history.history['acc'])
            grads.append(get_gradients_norm(self.model, x_train, y_train))
            
            if(cnt%collect_interval == 0):
                w = compressed_weights(self.model)
                weights.append(w)
            
        self.loss = loss
        self.accuracy = accuracy
        self.weights = weights
        self.grads = grads

In [5]:
def minimal_ratio(model, sample_amount, x, y, has_metrics=False):
    max_len = 1e-5
    weights = np.array(model.get_weights())
    config = model.get_config()
    pmodel = Model.from_config(config)
    if has_metrics:
        base_loss, metrics = model.evaluate(x=x, y=y)
    else:
        base_loss = model.evaluate(x=x, y=y)

    num_greater = 0

    for i in range(sample_amount):
        noise = np.random.rand(weights.size)
        noise_len = np.random.uniform(0.0, max_len)
        noise *=  (noise_len / np.linalg.norm(noise))
        noise.reshape(weights.shape)

        sample = weights + noise
        sample = [ a for a in sample ]

        pmodel.set_weight(sample)
        if has_metrics:
            loss, metrics = pmodel.evaluate(x=x, y=y)
        else:
            loss = pmodel.evaluate(x=x, y=y)
        if loss > base_loss:
            num_greater += 1
    return num_greater / sample_amount

In [6]:

def get_weight_grad(model, inputs, outputs):
    """ Gets gradient of model for given inputs and outputs for all weights"""
    grads = model.optimizer.get_gradients(model.total_loss, model.trainable_weights)
    symb_inputs = (model._feed_inputs + model._feed_targets + model._feed_sample_weights)
    f = K.function(symb_inputs, grads)
    x, y, sample_weight = model._standardize_user_data(inputs, outputs)
    output_grad = f(x + y + sample_weight)
    return output_grad


In [7]:

def compressed_weights(model):
    model_weights = np.array(model.get_weights())
    #print(DNN_weights)
    comp = []
    for i in range(len(model_weights)):
        model_weights[i] = model_weights[i].reshape(len(model_weights[i]),-1)
        model_weights[i] = model_weights[i].flatten()
        model_weights[i] = model_weights[i].reshape(len(model_weights[i]),-1)
        #print(model_weights[i].shape)
        for j in range(len(model_weights[i])):
            comp.append(model_weights[i][j])
    comp = np.array(comp).flatten()
    return comp
    

In [8]:

def get_gradients_norm(model, inputs, outputs):
    #from https://stackoverflow.com/questions/51140950/how-to-obtain-the-gradients-in-keras
    """ Gets gradient of model for given inputs and outputs for all weights"""
    grads = model.optimizer.get_gradients(model.total_loss, model.trainable_weights)
    symb_inputs = (model._feed_inputs + model._feed_targets + model._feed_sample_weights)
    f = K.function(symb_inputs, grads)
    x, y, sample_weight = model._standardize_user_data(inputs, outputs)
    output_grad = f(x + y + sample_weight)
    grad_sum = 0
    for i in range(len(output_grad)):
        grad_sum += np.sum(output_grad[i])**2
    grad_norm = grad_sum ** 0.5
    return grad_norm


In [37]:
if __name__ =='__main__':
  
    x_train, y_train = trainingData()
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
    batch_size = len(x_train)
    
    
    #find hessian reference to  https://gist.github.com/guillaume-chevalier/6b01c4e43a123abf8db69fa97532993f 
    # Input and Output. No batch_size for simplicity.
    x = tf.placeholder(shape=[batch_size], dtype=tf.float32, name="x")
    y = tf.placeholder(shape=[batch_size], dtype=tf.float32, name="y")
    
    W = tf.Variable(tf.ones(shape=[1]), dtype=tf.float32, name="W")
    b = tf.Variable(tf.zeros(shape=[1]), dtype=tf.float32, name="b")
    

    
    pred = tf.nn.relu(x * W + b)
    # my designed MSE
    loss = (tf.reduce_mean( (y - pred)**2) * batch_size) ** 0.5
    
    
    #make a shallow model to count gradient and hessian easier    
        
    wrt_variables = [W, b]
 
    grads = tf.gradients(loss, wrt_variables)

    hess = tf.hessians(loss, wrt_variables)
    inv_hess = [tf.matrix_inverse(h) for h in hess]

    #newton method, make the learning rate equalt to inverse Hessian
    update_directions = [
    - tf.reduce_sum(h) * g
    for h, g in zip(inv_hess, grads)
    ]
    op_apply_updates = [
    v.assign_add(up)
    for v, up in zip(wrt_variables, update_directions)
    ]


    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    eigens = 0;
    local_min = 0;
    initial_loss = sess.run(
        loss,
        feed_dict={
            x: x_train,
            y: y_train
        }
    )

    # Weight and bias update, "training" phase:
    for epoch in range(1000):
        new_loss, gradients, temp = sess.run(
            [loss, grads, op_apply_updates
            ],
            feed_dict={
                x: x_train,
                y: y_train
            }
        )
        print("weight:%f bias:%f"%(temp[0],temp[1]))
        grads_norm = 0
        for i in range(len(gradients)):
            grads_norm += gradients[i]**2
        grads_norm ** 0.5
        
        print("epoch:{} Loss:{}".format(epoch, new_loss))
        print("grads_norm:%f"%grads_norm)
        if(grads_norm > -0.0001 and grads_norm < 0.0001):
            eigenvalues, eigenvectors = sess.run(tf.self_adjoint_eig(A))

            for i in eigenvalues:
                if t > 0:
                    local_min = local_min + 1
                eigens = eigens + 1
            print("minimim ratio:%f", local_min/eigens)



weight:-0.925838 bias:-0.882021
epoch:0 Loss:60.77377700805664
grads_norm:7034.338867


InvalidArgumentError: Input is not invertible.
	 [[node MatrixInverse_80 (defined at <ipython-input-37-b1cc9e63d004>:30)  = MatrixInverse[T=DT_FLOAT, adjoint=false, _device="/job:localhost/replica:0/task:0/device:CPU:0"](Reshape_161)]]

Caused by op 'MatrixInverse_80', defined at:
  File "/home/marvin/anaconda3/envs/ML_env/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/marvin/anaconda3/envs/ML_env/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/marvin/anaconda3/envs/ML_env/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/marvin/anaconda3/envs/ML_env/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/marvin/anaconda3/envs/ML_env/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/home/marvin/anaconda3/envs/M